In [12]:
from __future__ import print_function
import pandas as pd
#import modin.pandas as pd
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import CuDNNLSTM
from keras.layers import BatchNormalization
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

import math
import os
import time


from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import music21
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
# load note coding labels 
labels = pd.read_csv('output/labelled_notes.csv')
labels = labels.values.tolist()

print('total labels:', len(labels))

total labels: 461


In [10]:
df = pd.read_csv('output/processed_output.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

df

,song,a1-0.125,a1-0.25,a1-0.3333333333333333,a1-0.5,a1-0.75,a1-1.0,a1-1.5,a1-2.0,a2-0.125,...,x-0.125,x-0.16666666666666666,x-0.25,x-0.3333333333333333,x-0.5,x-0.6666666666666666,x-0.75,x-1.0,x-1.5,x-2.0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476657,524,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
476658,524,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
476659,524,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
476660,524,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [11]:
#def process_row(row):

maxlen = 32
step = 1
dim = len(labels)
#df = df[df['song'] == 1]

def old_create_batches():
    sequences = np.zeros(shape=(len(df), maxlen, dim), dtype=np.int8)
    next_notes = np.zeros(shape=(len(df), dim), dtype=np.int8)

    songs_count = len(df['song'].unique())

    batch_id = 0
    for song_idx in range(1, songs_count):
        df_part = df[df['song'] == song_idx]
        df_part.drop(columns=['song'], inplace=True)
        length = len(df_part) - maxlen - 1
    
        for slice_start in range(0, length, step):
            sample_id = 0
            for row in df_part[slice_start: maxlen + slice_start].iterrows():
            
                sequences[batch_id][sample_id] = row[1]
            
                sample_id += 1
            next_notes[batch_id] = row[1]
            batch_id += 1
            
    return sequences, next_notes, songs_count

            
def create_batches(): #94% better perf
    sequences = np.zeros(shape=(len(df), maxlen, dim), dtype=np.int8)
    next_notes = np.zeros(shape=(len(df), dim), dtype=np.int8)

    songs_count = len(df['song'].unique())

    batch_id = 0
    for song_idx in range(1, songs_count):
        df_part = df[df['song'] == song_idx]
        df_part.drop(columns=['song'], inplace=True)
        bucket_count = len(df_part) - maxlen - 1
    
        for slice_start in range(0, bucket_count, step):

            next_notes[batch_id] = df_part.iloc[maxlen + slice_start + 1]
            sequences[batch_id] = df_part[slice_start: maxlen + slice_start].to_numpy()
            batch_id += 1
            
    return sequences, next_notes, songs_count



t1 = time.time()
sequences, next_notes, songs_count = create_batches()

t2 = time.time()
print (t2 - t1)  

#t1 = time.time()
#sequences, next_notes, songs_count = old_create_batches()

#t2 = time.time()
#print (t2 - t1)  

print('nb songs: ', songs_count)
print('nb sequences:', len(sequences))
print('nb next_notes:', len(next_notes))

To request implementation, send an email to feature_requests@modin.org.


RayTaskError: [36mray_worker[39m (pid=31130, host=KaOS)
ray.exceptions.RayTaskError: [36mray_worker[39m (pid=31149, host=KaOS)
  File "/home/jasam/anaconda3/lib/python3.7/site-packages/ray/memory_monitor.py", line 106, in raise_if_low_memory
    self.error_threshold))
ray.memory_monitor.RayOutOfMemoryError: More than 95% of the memory on node KaOS is used (32.01 / 33.69 GB). The top 10 memory consumers are:

PID	MEM	COMMAND
30797	9.59GB	/home/jasam/anaconda3/bin/python -m ipykernel_launcher -f /home/jasam/.local/share/jupyter/runtime/k
10246	1.65GB	/home/jasam/__SOFT/idea-IU-192.6817.14/jbr/bin/java -classpath /home/jasam/__SOFT/idea-IU-192.6817.1
1531	0.33GB	/usr/lib/chromium/chromium --type=gpu-process --field-trial-handle=5981818142672119073,1172871612533
31106	0.3GB	/home/jasam/anaconda3/lib/python3.7/site-packages/ray/core/src/ray/raylet/raylet --raylet_socket_nam
31099	0.3GB	/home/jasam/anaconda3/lib/python3.7/site-packages/ray/core/src/ray/thirdparty/redis/src/redis-server
1

In [ ]:
import tensorflow_addons as tfa
labels_count = len(labels)

model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, labels_count)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=True, activation=tfa.activations.gelu))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=False, activation=tfa.activations.gelu))
model.add(Dense(512, activation=tfa.activations.gelu))
model.add(Dropout(0.3))
model.add(Dense(labels_count, activation='softmax'))

 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc', 'categorical_crossentropy'])

In [ ]:
#model 2
import tensorflow_addons as tfa
labels_count = len(labels)

model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, labels_count)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(512, return_sequences=True, activation="relu")))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=False, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(labels_count, activation='softmax'))

 
model.compile(loss='categorical_crossentropy', optimizer='adam')
print('model 2')

In [ ]:
#model 3


In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def decode(index):
    return labels[index][0]

def sequence_to_label(sequence):
    result = []
    for index, data in sequence.iterrows():
        for label, content in data.items():
            if content > 0 :
                result.append(label)
    return result

def export_to_midi(string, name):
    notes = string.strip().split(' ')
    s = music21.stream.Stream()
    for row in notes:
        data = row.split("-")
        note = data[0]
        duration= float(data[1])
    
        if not note == 'x':
            if note == 'r':
                s.append(music21.note.Rest(quarterLength=duration))
            else:
                s.append(music21.note.Note(note, quarterLength=duration))

    print(s)
    s.write("midi", 'predicted/' + name)
    
def get_file_name(epoch, diversity):
    now = datetime.datetime.now()
    return now.strftime("%Y%m%d%H%M") + '_' +str(epoch) +'_' + str(diversity) + '.mid'
    
def get_seed():
    return df_copy[start_index: start_index + maxlen]

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(df) - maxlen - 1)
    start_index = 128
    for diversity in [0.2,0.3,0.4,0.5,0.6]:
        print('----- diversity:', diversity)
        
        df_copy = df.drop(columns=['song'])
        sequence = df_copy[start_index: start_index + maxlen]
        
        #print('----- Generating with seed: "' + ','.join(sequence_to_label(sequence)) + '"') 
        #print()
        predicted = ''
        for i in range(1024):
            # czary mary
            sequences = np.zeros(shape=(1, maxlen, len(labels)), dtype=float)
            np.append(sequences, sequence)
            x_pred = sequences

            # predicting 
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            
            
            next_char = decode(next_index)

            #adding new note TODO
            new_note = np.zeros(shape=(len(labels)), dtype=float)
            new_note[next_index] = 1
            
            sequence = sequence[1:]
            sequence.append(pd.DataFrame(new_note), ignore_index=True)
            
            #add
            predicted += next_char + ' '
            #writing
            sys.stdout.write(next_char + ' ')
            sys.stdout.flush()
            
        export_to_midi(predicted, get_file_name(epoch, diversity))
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
#fit_generator(generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, validation_freq=1, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

history = model.fit(sequences, next_notes,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

#shuffle??

In [10]:
import tensorflow_addons as tfa
from keras.layers import Conv2D, MaxPooling2D, Input
labels_count = len(labels)
from keras.models import Model

inputs = Input(shape=(maxlen, labels_count),  name='inputs')


#model1 = Sequential()
#model1.add(LSTM(512, return_sequences=True, input_shape=(maxlen, labels_count)))
#model1.add(Dropout(0.3))
#model1.add(LSTM(512, return_sequences=True, activation="relu"))
#model1.add(Dropout(0.3))
#model1.add(LSTM(512, return_sequences=False, activation="relu"))

x1 = LSTM(512, return_sequences=True, input_shape=(maxlen, labels_count))(inputs)
model1_out = Dense(labels_count, activation='softmax')(x1)

#model2 = Sequential()
#model2.add(LSTM(512, return_sequences=True, input_shape=(maxlen, labels_count)))
#model2.add(Dropout(0.3))
#model2.add(LSTM(512, return_sequences=True, activation=tfa.activations.gelu))
#model2.add(Dropout(0.3))
#model2.add(LSTM(512, return_sequences=False, activation=tfa.activations.gelu))

x2 = LSTM(512, return_sequences=True, input_shape=(maxlen, labels_count))(inputs)
model2_out = Dense(labels_count, activation='softmax')(x2)


model = Model(inputs=inputs, outputs=[model1_out, model2_out])

model.compile(loss='categorical_crossentropy', optimizer='adam', loss_weights=[1., 0.5])

history = model.fit(sequences, [next_notes, next_notes],
          batch_size=128,
          epochs=10)

df_copy = df.drop(columns=['song'])
sequence = df_copy[start_index: start_index + maxlen]


ValueError: Error when checking target: expected dense_7 to have 3 dimensions, but got array with shape (476662, 461)

In [ ]:
for label in ["acc"]:
    plt.plot(history.history[label],label=label)
plt.legend()
plt.show()

for label in ["loss"]:
    plt.plot(history.history[label],label=label)
plt.legend()
plt.show()